In [ ]:
%reload_ext autoreload
%autoreload 2


In [ ]:
import sys

In [ ]:


# sys.path.insert(0,'E:/repos/my_projects/taxi_demand_project')
sys.path.insert(0,'E:/repos/my_projects/taxi_demand_project/src/')
sys.path.insert(0,'E:/repos/my_projects/taxi_demand_project')


In [ ]:
sys.path

In [ ]:
from src.data import print_data1_message, download_data

In [ ]:
print_data1_message('hi!')

In [ ]:
from data2 import print_data2

In [ ]:
print_data2('data2 is working')

In [25]:
from src.paths import RAW_DATA_DIR
import requests
# import pandas as pd
# from typing import Optional , List
# from tqdm import tqdm
from pathlib import Path

def download_data_test(year:int , month:int) -> Path:

    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    print('the url is ',url)
    response = requests.get(url)

    if response.status_code == 200:
        path = RAW_DATA_DIR / f'taxi_{year}-{month:02d}.parquet'
        open(path,'wb').write(response.content)
        return path
    else:
        raise Exception(f'{url} not available') 
    


In [26]:
z = download_data_test(2022,1)

the url is  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet


In [2]:
import sys
sys.path.insert(0, 'E:/repos/my_projects/taxi_demand_project')


In [3]:
%reload_ext autoreload
%autoreload 2
from src.data import load_raw_data, transform_to_ts_data
raw = load_raw_data(year=2022, months = [1,2])


file taxi_2022_01.parquet already downloaded
file taxi_2022_02.parquet already downloaded


In [5]:
raw['PU_datetime']

Timestamp('2003-01-01 00:10:06')

In [7]:
raw['PU_hour']=raw['PU_datetime'].dt.floor('H')

In [9]:
raw['PU_hour'].min()

Timestamp('2003-01-01 00:00:00')

In [26]:
year = 2022
month = 1
start = f'{year}-{month:02d}-01'
end = f'{year+1}-{month:02d}-01'



In [27]:
dates= raw[raw['PU_hour'] >= start]
dates = dates[dates['PU_hour']< end] 

In [28]:
dates['PU_hour'].max()

Timestamp('2022-02-28 23:00:00')

In [1]:
import sys
sys.path.insert(0,'E:/repos/NYC_tax_demand_predictor')

import streamlit as st
from datetime import datetime , timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pydeck as pdk
import requests
import zipfile

from src.plot import plot_one_example
from src.paths import DATA_DIR
from src.inference import (
    load_model_form_registry,
    load_model_from_computer,
    get_batch_data_from_feature_store,
    get_model_predictions
)
from src.data import(load_raw_data , 
                     transform_to_ts_data,
                     )
from src.load_data_from_computer import load_features_from_computer 


st.set_page_config(layout='wide')


st.title('Taxi Demand Predictor')

time_now = pd.to_datetime(datetime.utcnow()).floor('H')

st.header(f'Time Now {time_now}')


progress_bar =  st.sidebar.header('Work in progress')
progress_bar = st.sidebar.progress(0)

N_steps = 7


def load_shape_data():
    URL = 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'
    response = requests.get(URL)
    path = DATA_DIR / 'taxi_zones/taxi_zone.zip'
    if response.status_code ==200:
        open(path,'wb').write(response.content)
    else:
        print(f'The {URL} is not available')
    
    with zipfile.ZipFile(path,'r') as zip:
        zip.extractall(DATA_DIR/'taxi_zones')
    
    return gpd.read_file(DATA_DIR/'taxi_zones/taxi_zones.shp').to_crs('epsg:4326')

# with st.spinner(text ='Fetching the most inference data'):
#     features =  get_batch_data_from_feature_store(current_time=time_now)
#     st.spinner.write('Inference data retrived')
#     progress_bar.progress(2/N_steps)
#     print(f'{features}')



  `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
  of the deprecation of `distutils` itself. It will be removed for
  Python >= 3.12. For older Python versions it will remain present.
  It is recommended to use `setuptools < 60.0` for those Python versions.
  For more details, see:
    https://numpy.org/devdocs/reference/distutils_status_migration.html 


2023-07-02 16:35:17.502 
  command:

    streamlit run e:\repos\NYC_tax_demand_predictor\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [2]:
with st.spinner(text= 'Downloading shape file for plotting taxi zones'):
     gpd_pd = load_shape_data()
     st.sidebar.write('The shape file is downloaded')
     progress_bar.progress(1/N_steps)

In [3]:
with st.spinner(text ='Fetching the inference data'):
    features =  load_features_from_computer(current_time= time_now)
    st.sidebar.write('Inference data retrived')
    progress_bar.progress(2/N_steps)
    print(f'{features}')

current time 2023-07-02 08:00:00
file taxi_2022_06.parquet already downloaded.
file taxi_2022_07.parquet already downloaded.


100%|██████████| 261/261 [00:02<00:00, 100.06it/s]


fetching data from 2023-06-04 08:00:00 to 2023-07-02 07:00:00
     rides_672_hr_before  rides_671_hr_before  rides_670_hr_before  \
0                    1.0                  2.0                  0.0   
1                    1.0                  4.0                  2.0   
2                    2.0                  1.0                  5.0   
3                    1.0                  0.0                  0.0   
4                   13.0                 19.0                 39.0   
..                   ...                  ...                  ...   
256                  0.0                  0.0                  0.0   
257                  0.0                  0.0                  0.0   
258                  0.0                  0.0                  0.0   
259                  0.0                  0.0                  0.0   
260                  0.0                  0.0                  0.0   

     rides_669_hr_before  rides_668_hr_before  rides_667_hr_before  \
0                    0.0   

In [4]:
with st.spinner(text='Loading model from Model Registry'):
    model = load_model_from_computer()
    st.sidebar.write('Model loaded from Registry')
    progress_bar.progress(3/N_steps)


In [5]:
with st.spinner(text='Loading Prediction from Model Registry'):
    results = get_model_predictions(model, features)
    st.sidebar.write('Predictions loaded from Registry')
    progress_bar.progress(4/N_steps)


In [27]:
def color_scaling(x,minval,maxval,start_color,end_color):
    f = float(x-minval)/(maxval-minval)
    scale = tuple(f*(b-a)+a for (a, b) in zip(start_color, end_color))
    return scale

df = pd.merge(gpd_pd, results, 
    right_on = 'pu_location',left_on = 'LocationID', how ='inner')

BLACK, GREEN = (0,0,0), (0,255,0)

df['color_scale'] = df['predicted_demand']
max_value, min_value = df['color_scale'].max(), df['color_scale'].min()

df['color'] = df['color_scale'].apply(lambda x : color_scaling(x,min_value,
                                                                max_value,BLACK,GREEN))
   
   


In [28]:
df.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough',
       'geometry', 'pu_location', 'predicted_demand', 'color_scale', 'color'],
      dtype='object')

In [31]:
df[['LocationID','pu_location','predicted_demand','color_scale','color']]

,LocationID,pu_location,predicted_demand,color_scale,color
0,1,1,1.0,1.0,"(0.0, 1.795774647887324, 0.0)"
1,2,2,0.0,0.0,"(0.0, 0.0, 0.0)"
2,3,3,0.0,0.0,"(0.0, 0.0, 0.0)"
3,4,4,2.0,2.0,"(0.0, 3.591549295774648, 0.0)"
4,5,5,0.0,0.0,"(0.0, 0.0, 0.0)"
...,...,...,...,...,...
254,259,259,0.0,0.0,"(0.0, 0.0, 0.0)"
255,260,260,0.0,0.0,"(0.0, 0.0, 0.0)"
256,261,261,10.0,10.0,"(0.0, 17.95774647887324, 0.0)"
257,262,262,50.0,50.0,"(0.0, 89.78873239436619, 0.0)"
